In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import pymysql
import datetime
from prophet import Prophet

In [18]:
data = pd.read_csv('./돼지고기가격.csv',index_col=0)

In [19]:
data

,date,도매가격
1714,2016-01-04,4221.0
1713,2016-01-05,4083.0
1712,2016-01-06,4117.0
1711,2016-01-07,4148.0
1710,2016-01-08,4242.0
...,...,...
3,2022-11-14,5685.0
2,2022-11-15,5619.0
1,2022-11-16,5601.0
0,2022-11-17,5653.0


In [20]:
data['date'] = pd.to_datetime(data['date'])

In [21]:
mydb = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '1234',
    db = 'price',
    charset='utf8'
)

cursor = mydb.cursor(pymysql.cursors.DictCursor)
cursor.execute("SELECT * FROM price.pork ORDER BY 1 DESC LIMIT 5")
result = cursor.fetchall()

for row in result :
    print(row)

{'date': '2022-11-25', '도매가격': 5710}
{'date': '2022-11-24', '도매가격': 5604}
{'date': '2022-11-23', '도매가격': 5577}
{'date': '2022-11-22', '도매가격': 5602}
{'date': '2022-11-21', '도매가격': 5680}


In [22]:
# 조회 결과 데이터프레임으로 저장
df = pd.DataFrame(result)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date',ascending=True)
df

,date,도매가격
4,2022-11-21,5680
3,2022-11-22,5602
2,2022-11-23,5577
1,2022-11-24,5604
0,2022-11-25,5710


In [23]:
days_1 = pd.DataFrame(df['date'][-1:]  + datetime.timedelta(days=1))
days_2 = pd.DataFrame(df['date'][-1:]  + datetime.timedelta(days=2))
days = pd.concat((days_1,days_2))
days

,date
0,2022-11-26
0,2022-11-27


In [24]:
df = pd.merge(df,days,on='date',how='outer')
df = df.fillna(method='ffill')
df

,date,도매가격
0,2022-11-21,5680.0
1,2022-11-22,5602.0
2,2022-11-23,5577.0
3,2022-11-24,5604.0
4,2022-11-25,5710.0
5,2022-11-26,5710.0
6,2022-11-27,5710.0


In [26]:
data = pd.concat((data,df),axis=0)
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by='date', ascending=True)
data = data.drop_duplicates(subset=['date'], keep='first')
data = data[-2220:]
data

,date,도매가격
1514,2016-10-28,4141.0
1813,2016-10-29,4141.0
1814,2016-10-30,4141.0
1513,2016-10-31,4490.0
1512,2016-11-01,4612.0
...,...,...
2,2022-11-23,5577.0
3,2022-11-24,5604.0
4,2022-11-25,5710.0
5,2022-11-26,5710.0


In [27]:
# prophet은 ds,y라는 두가지 컬럼만 있어야 함(컬럼 이름 맞추기)
data = data[['date', '도매가격']]
data = data.rename(columns=({'date':'ds','도매가격':'y'}))

In [31]:
from prophet import Prophet

m = Prophet(changepoint_range=0.9,changepoint_prior_scale=0.25)
# m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
m.add_country_holidays(country_name='KR').fit(data)
future = m.make_future_dataframe(periods=30,freq='D') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

11:38:59 - cmdstanpy - INFO - Chain [1] start processing
11:39:00 - cmdstanpy - INFO - Chain [1] done processing


In [35]:
forecast =  forecast[['ds','yhat']][-30:]
forecast

,ds,yhat
2220,2022-11-28,6310.251778
2221,2022-11-29,6294.857639
2222,2022-11-30,6277.899829
2223,2022-12-01,6269.805930
2224,2022-12-02,6215.505000
2225,2022-12-03,6213.911436
2226,2022-12-04,6205.229743
2227,2022-12-05,6317.181250
2228,2022-12-06,6273.305505
2229,2022-12-07,6226.787697
